In [2]:
import os
from ultralytics import YOLO
import supervision as sv
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [35]:

path = os.path.join(os.getcwd(), 'datasets', 'detection_dataset_new_robo_top_1class')
print(path)

d:\Programmation\TPS\sew\NEW\datasets\detection_dataset_new_robo_top_1class


: 

In [30]:
# Load a model
# model = YOLO(os.path.join('guide_results', 'DS', 'weights', 'best.pt'))  # load a pretrained model (recommended for training)

model = YOLO("yolov8n")
# Train the model
model.train(cfg='cfg_det.yaml', data = os.path.join(path, 'data.yaml'), name = 'top_detection', batch = 2, epochs = 10, iou = 0.8)

100%|██████████| 6.25M/6.25M [00:00<00:00, 11.1MB/s]


New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.59  Python-3.12.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=d:\Programmation\TPS\sew\NEW\datasets\detection_dataset_new_robo_top_1class\data.yaml, epochs=10, time=None, patience=20, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=6, project=guide_results, name=top_detection5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.8, max_det=300, half=False, dnn=False, plots=True, source=

train: Scanning D:\Programmation\TPS\sew\NEW\datasets\detection_dataset_new_robo_top_1class\train\labels.cache... 126 images, 0 backgrounds, 0 corrupt: 100%|██████████| 126/126 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
WARNING  'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False


# Inference

In [3]:
# upload model to roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="PfgdGiZ3LwFyIZrOhgl7")
project = rf.workspace("cilcare").project("topboxdetection")
version = project.version(4)
# dataset = version.download("yolov8")
                

model = YOLO("guide_results/top_detection4/weights/best.pt")



loading Roboflow workspace...
loading Roboflow project...


In [7]:
version.deploy("yolov8", r"D:\Programmation\TPS\sew\NEW\guide_results\top_detection4\weights", "best.pt")

Dependency ultralytics==8.0.196 is required but found version=8.3.59, to fix: `pip install ultralytics==8.0.196`
View the status of your deployment at: https://app.roboflow.com/cilcare/topboxdetection/4
Share your model with the world at: https://universe.roboflow.com/cilcare/topboxdetection/model/4


: 

In [1]:
# Inference for top view

model = YOLO("guide_results/top_detection4/weights/best.pt")
img_path = r"../data/new/test/top_right.png"
frame = cv2.imread(img_path)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# resizes the image to the model's image size or a bit more
frame = cv2.resize(frame, (640, 640))
frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

results = model(frame)[0]
box_annotator = sv.BoxAnnotator(thickness=1,)
# label_annotator = sv.LabelAnnotator()
detections = sv.Detections.from_ultralytics(results)

# Filter detections with confidence
mask = detections.confidence > 0.6
detections = detections[mask]

labels = [
    f"{model.names[class_id]} {confidence:0.2f}"
    for confidence, class_id in zip(detections.confidence, detections.class_id)
]

# Annotate the image
annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
# annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
# show total count
count = len(detections)
annotated_frame = cv2.putText(annotated_frame, f"Count: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

plt.figure(figsize=(12, 8))
plt.imshow(annotated_frame)
plt.axis('off')
plt.show()

NameError: name 'YOLO' is not defined